In [ ]:
!pip install roboflow
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
from roboflow import Roboflow
import cv2
import os
from dotenv import load_dotenv
load_dotenv() 


True

In [ ]:
rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
project = rf.workspace("mikhails-thesis").project("photo-album-processing-a1imo")
version = project.version(2)
dataset = version.download("yolov8-obb")
                

In [ ]:


model = YOLO("yolov8m-obb.pt")  # Use yolov8n-obb.pt, yolov8s-obb.pt, etc.

model.train(data=dataset.location + "/data.yaml", epochs=200, imgsz=640, val=True)

In [ ]:
model = YOLO("/Users/mkhlrmnv/Desktop/kuva-prosessointi/custom_ml/finetuned-yolov8m-obb-200epoch.pt")

res = model.predict("/Users/mkhlrmnv/Desktop/kuva-prosessointi/kuva-processointi/photo-album-processing-2/train/images/Skannaus-3_jpeg.rf.63c1e7bc3b16129091b72ffad8c105a7.jpg", conf=0.4)

cv2.imshow("img", res[0].plot())
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 /Users/mkhlrmnv/Desktop/kuva-prosessointi/kuva-processointi/photo-album-processing-2/train/images/Skannaus-3_jpeg.rf.63c1e7bc3b16129091b72ffad8c105a7.jpg: 640x480 None215.9ms
Speed: 1.7ms preprocess, 215.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)


: 

In [ ]:
import glob
import os

# Load the model
model = YOLO("/Users/mkhlrmnv/Desktop/kuva-prosessointi/custom_ml/finetuned-yolov8m-obb-200epoch.pt")

# Define the folder path containing images
image_folder = "/Users/mkhlrmnv/Desktop/kuva-prosessointi/kuva-processointi/photo-album-processing-2/train/images/"

# Supported image extensions
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff', '*.tif']

# Get all image files in the folder
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(image_folder, ext)))
    image_files.extend(glob.glob(os.path.join(image_folder, ext.upper())))

print(f"Found {len(image_files)} images in the folder")

# Process each image
for i, image_path in enumerate(image_files):
    print(f"Processing image {i+1}/{len(image_files)}: {os.path.basename(image_path)}")
    
    # Make prediction
    results = model.predict(image_path, conf=0.4)
    
    # Get the plotted image with bounding boxes
    plotted_img = results[0].plot()
    
    # Display the image
    cv2.imshow(f'Detection Results - {os.path.basename(image_path)}', plotted_img)
    
    # Wait for key press - press 'q' to quit, any other key to continue
    key = cv2.waitKey(0) & 0xFF
    cv2.destroyAllWindows()
    
    if key == ord('q'):
        print("Stopping processing...")
        break
    
    print(f"Detected {len(results[0].boxes)} objects in {os.path.basename(image_path)}")

print("Processing complete!")

In [ ]:
# Alternative: Batch process all images without display
# Useful for collecting statistics or saving results

image_folder = "/Users/mkhlrmnv/Desktop/kuva-prosessointi/kuva-processointi/photo-album-processing-2/train/images/"
output_folder = "/Users/mkhlrmnv/Desktop/kuva-prosessointi/output/"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get all image files
image_files = []
for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp']:
    image_files.extend(glob.glob(os.path.join(image_folder, ext)))

detection_stats = []

for i, image_path in enumerate(image_files):
    filename = os.path.basename(image_path)
    print(f"Processing {i+1}/{len(image_files)}: {filename}")
    
    # Make prediction
    results = model.predict(image_path, conf=0.4, save=False)
    
    # Collect statistics
    num_detections = len(results[0].boxes) if results[0].boxes is not None else 0
    detection_stats.append({
        'filename': filename,
        'detections': num_detections,
        'confidence_scores': results[0].boxes.conf.tolist() if results[0].boxes is not None else []
    })
    
    # Optional: Save annotated image
    if num_detections > 0:
        annotated_img = results[0].plot()
        output_path = os.path.join(output_folder, f"annotated_{filename}")
        cv2.imwrite(output_path, cv2.cvtColor(annotated_img, cv2.COLOR_RGB2BGR))

print(f"\nProcessing complete! Statistics:")
print(f"Total images processed: {len(image_files)}")
print(f"Images with detections: {sum(1 for stat in detection_stats if stat['detections'] > 0)}")
print(f"Total detections: {sum(stat['detections'] for stat in detection_stats)}")

# Show detailed stats
for stat in detection_stats[:5]:  # Show first 5 results
    print(f"{stat['filename']}: {stat['detections']} detections")